# Setup

In [1]:
%matplotlib inline
from pylab import *
import math

import detectorgeometry_trap as geom_trap
reload(geom_trap)

import decay_properties as decay
reload(decay)

<module 'decay_properties' from 'decay_properties.pyc'>

In [ ]:
# example data input to class to determine effective area
theta_src = radians(83) #radians
phi_src = radians(180)
h = 35.5 #km
R = 6356. # km
ice =4
theta_view = radians(1) #degrees
N = 2000000 # number of samples
E_nu = 18 #log energy in eV of incident neutrino 
E_cut = 14

geom_area = geom_trap.Area(theta_src,phi_src, h,R,theta_view, N)
A_deg, phi_e, theta_e, view_angle, exit_angle, norm, e_dot = geom_area.event_retention()
print A_deg

In [ ]:
# Assign to each event a tau energy, tau decay point, a decay probability, calculated view angle
# inputs : theta_src, ice, E_cut, e_dot, phi_e, t_e, rho, E_nu,h,R,view_cut, exit_angles
decay1 = decay.tau_event(theta_src, ice, E_cut, e_dot, phi_e, theta_e, norm,E_nu, h, R, theta_view, exit_angle, view_angle, A_deg)
                        

In [ ]:
# ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_exit_obs,ret_exit_decay, ret_decay_obs, ret_exit,\
# ret_view, ret_decay_angle, ret_fractions,ret_types, ret_p_exit = decay1.event_retention()

A_ret,ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_exit_obs, ret_exit_decay, ret_decay_obs, \
ret_exit, ret_view, ret_decay_alt, ret_decay_angle, ret_fractions,ret_types, ret_p_exit = decay1.degree_eff_area()

# print A_ret, len(ret_e_dot)
# print ret_exit_obs,ret_exit_decay, ret_decay_obs

## Adding RF cuts

In [ ]:
f_lo = 180
f_high = 1200
Gain_dB = 10
Nphased = 1

In [2]:
import trigger_properties as RF
reload(RF)

<module 'trigger_properties' from 'trigger_properties.pyc'>

In [ ]:
trigger_result = RF.event_detection(A_ret, f_lo, f_high, Gain_dB, Nphased, 
                                    ret_exit,ret_view, ret_decay_angle, ret_decay_alt,
                                    theta_view, ret_exit_obs,ret_exit_decay, ret_decay_obs,
                                    ret_tau_energy, R, ice, h, ret_t_e, ret_phi_e,theta_src,phi_src)

In [ ]:
trigger_result.RF_eff_area()

## Creation of Graphs with Successive Effective Area Refinement

In [8]:
phi_src = radians(180)
h = 35.5 #km
R = 6356. # km
ice =4
theta_view = radians(1) #degrees
N = 2000000 # number of samples
E_nu = 18 #log energy in eV of incident neutrino 
E_cut = 14
f_lo = 180
f_high = 1200
Gain_dB = 10
Nphased = 1

In [10]:


# Creating a plot for the full range of possible source declinations 
start = 0 # minimum source declination in degrees
end = 90 # maximum source declination in degrees
steps=90 # divisions indeclination used to generate plot
x1 = np.linspace(-end,-start,steps)

A_g = []
A_p=[]
A_rf = []
theta_view = radians(1)
for j in range(0,steps):
    print j
    theta_src = radians(start+(end-start)*float(j)/steps)
    
    # GEOMETRIC
    geom_area = geom_trap.Area(theta_src,phi_src, h,R,theta_view, N)
    A_deg, phi_e, theta_e, view_angle, exit_angle, norm, e_dot = geom_area.event_retention()
    print A_deg
    
    # DECAY
    decay1 = decay.tau_event(theta_src, ice, E_cut, e_dot, phi_e, theta_e, norm,E_nu, h, R, theta_view, exit_angle, view_angle, A_deg)
    A_ret,ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_exit_obs, ret_exit_decay, ret_decay_obs, \
    ret_exit, ret_view, ret_decay_alt, ret_decay_angle, ret_fractions,ret_types, ret_p_exit = decay1.degree_eff_area()
    print A_ret
    # TRIGGER
    trigger_result = RF.event_detection(A_ret, f_lo, f_high, Gain_dB, Nphased, \
                                    ret_exit,ret_view, ret_decay_angle, ret_decay_alt, \
                                    theta_view, ret_exit_obs,ret_exit_decay, ret_decay_obs, \
                                    ret_tau_energy, R, ice, h, ret_t_e, ret_phi_e,theta_src,phi_src)
    
    A_trigger  = trigger_result.RF_eff_area()
    print A_trigger
    
    A_g.append(A_deg)
    A_p.append(A_ret)
    A_rf.append(A_trigger)




0
1.205493135894495


KeyboardInterrupt: 

In [ ]:
# Creating a plot for the full range of possible source declinations 
start = 0 # minimum source declination in degrees
end = 90 # maximum source declination in degrees
steps=90 # divisions indeclination used to generate plot
x1 = np.linspace(-end,-start,steps)

A_P_exit_energy = []
theta_view = radians(1)
for j in range(0,steps):
    print j
    theta_src = radians(start+(end-start)*float(j)/steps)
    # GEOMETRIC
    geom_area = geom_trap.Area(theta_src,phi_src, h,R,theta_view, N)
    A_deg, phi_e, theta_e, view_angle, exit_angle, norm, e_dot = geom_area.event_retention()
    print A_deg
    
    # DECAY
    decay1 = decay.tau_event(theta_src, ice, E_cut, e_dot, phi_e, theta_e, norm,E_nu, h, R, theta_view, exit_angle, view_angle, A_deg)
    A_ret,ret_e_dot, ret_phi_e, ret_t_e, ret_tau_energy, ret_exit_obs, ret_exit_decay, ret_decay_obs, \
    ret_exit, ret_view, ret_decay_alt, ret_decay_angle, ret_fractions,ret_types, ret_p_exit = decay1.degree_eff_area()

    A_P_exit_energy.append(A_ret)


In [ ]:
fig = plt.figure(figsize=(7,7))
ax = plt.axes(yscale='log')


ax.plot(x1, A_g)
ax.plot(x1, A_p)
ax.plot(x1, A_rf)


# ax.plot(x1, A_g)
# ax.plot(x1, A_P_exit)
# ax.plot(x1, A_P_exit_energy)
# ax.plot(x1, A_P_exit_energy_decay)
# ax.plot(x1, A_rf)
    
#plt.fill_between(x2, sky_maps[0],sky_maps[1],color='tab:gray')
horizon = degrees(np.arcsin(R / (R+h)))-90
plt.xlim(-90,0)  
legend = ax.legend([r'$\langle A \rangle_g$', 
                    #r'$\langle A \rangle_g \cdot P_{exit}$', 
                    #r'$\langle A \rangle_g \cdot P_{exit} \cdot P(E_\tau > 10^{14})$', 
                    r'$\langle A \rangle_g \cdot P_{exit} \cdot P(E_\tau > 10^{14}) \cdot P_{decay}$',
                   r'$\langle A \rangle_g \cdot P_{exit} \cdot P(E_\tau > 10^{14}) \cdot P_{decay} \cdot P_{trigger} $'],
                   fontsize=14, bbox_to_anchor=(1.1, 1))

plt.xlabel("Payload Elevation Angle of Source "+r'$\theta_{ele}$'+" (degrees)", fontsize=18)
plt.ylabel(r'$[km^2]$', fontsize=18)
plt.title("ANITA Effective Area", fontsize=20)
plt.setp(legend.get_title(),fontsize='x-large')
ax.tick_params(axis = 'both', which = 'major', labelsize = 16)

y_min =min(A_g)
y_max = max(A_g)
print y_min, y_max
y_ticks = [10**x for x in range(int(round(math.log10(10**-8))),int(round(math.log10(y_max))))]
plt.yticks(y_ticks)
ax.grid(True,which='both')
#plt.semilogy()
plt.show()

In [ ]:
trigger_result.RF_eff_area()

In [ ]:
print max(A_P_exit)